## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-03-02-25-12 +0000,missionlocal,‘I don’t want to come to school tomorrow:’ S.F...,https://missionlocal.org/2025/12/student-shot-...
1,2025-12-03-02-19-03 +0000,cbc,Trump says National Guard will soon go to New ...,https://www.cbc.ca/news/world/trump-new-orlean...
2,2025-12-03-02-17-00 +0000,wsj,China’s Services-Sector Activity Growth Slows,https://www.wsj.com/economy/central-banking/ch...
3,2025-12-03-02-11-38 +0000,nyt,U. of Alabama Suspends Black and Female Studen...,https://www.nytimes.com/2025/12/02/us/u-of-ala...
4,2025-12-03-02-07-51 +0000,latimes,"San Francisco sues Coca-Cola, Kellogg over ult...",https://www.latimes.com/california/story/2025-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2346/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,68
81,up,20
18,new,19
158,he,14
69,drug,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
72,2025-12-02-23-36-00 +0000,wsj,The Trump administration canceled interviews w...,https://www.wsj.com/economy/central-banking/tr...,214
7,2025-12-03-02-00-00 +0000,wsj,"Pete Hegseth, who won over Trump by defending ...",https://www.wsj.com/politics/national-security...,169
356,2025-12-02-03-20-00 +0000,wsj,Costco one of the biggest companies to sue the...,https://www.wsj.com/politics/policy/costco-joi...,137
18,2025-12-03-01-41-00 +0000,wsj,President Trump on Tuesday lashed out against ...,https://www.wsj.com/politics/policy/ice-plans-...,125
256,2025-12-02-14-41-48 +0000,nypost,Former Honduran President Juan Orlando Hernand...,https://nypost.com/2025/12/02/world-news/forme...,123


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
72,214,2025-12-02-23-36-00 +0000,wsj,The Trump administration canceled interviews w...,https://www.wsj.com/economy/central-banking/tr...
7,101,2025-12-03-02-00-00 +0000,wsj,"Pete Hegseth, who won over Trump by defending ...",https://www.wsj.com/politics/national-security...
358,56,2025-12-02-03-11-29 +0000,nyt,Kristi Noem Recommends New Travel Ban After Na...,https://www.nytimes.com/2025/12/01/us/politics...
102,53,2025-12-02-22-40-22 +0000,nypost,White House trolls Sabrina Carpenter with her ...,https://nypost.com/2025/12/02/us-news/white-ho...
192,49,2025-12-02-19-15-09 +0000,nypost,"Kraft, Coca-Cola among companies sued by San F...",https://nypost.com/2025/12/02/business/kraft-c...
204,48,2025-12-02-18-23-52 +0000,cbc,"Ex-Honduran president, convicted of traffickin...",https://www.cbc.ca/news/world/us-honduras-hern...
30,44,2025-12-03-01-11-00 +0000,wsj,Marathon Russia-U.S. Meeting Yields No Ukraine...,https://www.wsj.com/world/russia/trump-envoys-...
356,41,2025-12-02-03-20-00 +0000,wsj,Costco one of the biggest companies to sue the...,https://www.wsj.com/politics/policy/costco-joi...
52,39,2025-12-03-00-18-37 +0000,nypost,"NYC shootings hit all-time low, subways safer,...",https://nypost.com/2025/12/02/us-news/big-appl...
127,39,2025-12-02-21-52-49 +0000,nyt,Michael and Susan Dell Pledge $6 Billion in In...,https://www.nytimes.com/2025/12/02/business/de...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
